In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
paths = {
    "sales": "/content/drive/MyDrive/Cognizant/t3/sales.csv",
    "sensor_stock_levels": "/content/drive/MyDrive/Cognizant/t3/sensor_stock_levels.csv",
    "sensor_storage_temperature": "/content/drive/MyDrive/Cognizant/t3/sensor_storage_temperature.csv"
}

In [ ]:
def process_csv(csv_path):
    print(f"\nExtraction {csv_path}")
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)
    # Drop the "Unnamed: 0" column if it exists, and ignore errors if it doesn't
    df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
    # Return the modified DataFrame
    return df

In [ ]:
df_sales = process_csv(paths["sales"])
df_stock = process_csv(paths["sensor_stock_levels"])
df_temperature = process_csv(paths["sensor_storage_temperature"])


Extraction /content/drive/MyDrive/Cognizant/t3/sales.csv

Extraction /content/drive/MyDrive/Cognizant/t3/sensor_stock_levels.csv

Extraction /content/drive/MyDrive/Cognizant/t3/sensor_storage_temperature.csv


In [ ]:
#Seaborn visualization
!pip install seaborn
import seaborn as sns

#ANALYZING SALES

In [ ]:
#Sales
df_sales.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7829 non-null   object 
 1   timestamp       7829 non-null   object 
 2   product_id      7829 non-null   object 
 3   category        7829 non-null   object 
 4   customer_type   7829 non-null   object 
 5   unit_price      7829 non-null   float64
 6   quantity        7829 non-null   int64  
 7   total           7829 non-null   float64
 8   payment_type    7829 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 550.6+ KB


In [ ]:
df_sales.describe()

,unit_price,quantity,total
count,7829.000000,7829.000000,7829.000000
mean,7.819480,2.501597,19.709905
std,5.388088,1.122722,17.446680
min,0.190000,1.000000,0.190000
25%,3.990000,1.000000,6.570000
50%,7.190000,3.000000,14.970000
75%,11.190000,4.000000,28.470000
max,23.990000,4.000000,95.960000


In [ ]:
import random
import seaborn as sns

In [ ]:
def plot_continuous_distribution(data: pd.DataFrame = None, column: str = None, height: int = 8):
  _ = sns.displot(data, x=column, kde=True, height=height, aspect=height/5).set(title=f'Distribution of {column}');

def plot_categorical_distribution(data: pd.DataFrame = None, column: str = None, height: int = 8, aspect: int = 2):
  _ = sns.catplot(data=data, x=column, kind='count', height=height, aspect=aspect).set(title=f'Distribution of {column}');

display_plot = False
display_print = False


def stats_unique_values(data, column_name):
  column_rows = len(data[column_name])
  unique_values_count = len(data[column_name].unique())
  # Unique values
  if display_print == True:
      print(f"Column: {column_name} has {unique_values_count} unique values in {column_rows} rows\n")
  # for low volume data
  if unique_values_count < 50:
      # extract data
      value_counts = data[column_name].value_counts().sort_values(ascending=False)
      max_value_length = max(len(str(value)) for value in value_counts.index)
      total_count = value_counts.sum()
      for value, count in value_counts.items():
          percentage = (count / total_count) * 100
          if display_print == True:
              print(f'{value:{max_value_length}}  {count} ({percentage:.2f}%)')
      if display_print == True:
          print("")
      # Display plot
      if display_plot == True:
          if pd.api.types.is_numeric_dtype(data[column_name]):
              plot_continuous_distribution(data , column_name)
          else:
              plot_categorical_distribution(data , column_name)
  # for high volume data
  else:
      random_numbers = [random.randint(0, unique_values_count-1) for _ in range(5)]
      if display_print == True:
          print("Samples:")
      for num in random_numbers:
          if display_print == True:
              print(data[column_name][num])
      if pd.api.types.is_numeric_dtype(data[column_name]):
          column_stats = data[column_name].describe()
          # Extract and print specific statistics
          mean = column_stats['mean']
          std = column_stats['std']
          min_value = column_stats['min']
          quartile_25 = column_stats['25%']
          median = column_stats['50%']
          quartile_75 = column_stats['75%']
          max_value = column_stats['max']
          if display_print == True:
              print(f"\nDescription:")
              print(f"mean:{mean}")
              print(f"std: {std}")
              print(f"min: {min_value}")
              print(f"25%: {quartile_25}")
              print(f"50%: {median}")
              print(f"75%: {quartile_75}")
              print(f"max: {max_value}")
      else:
          if display_print == True:
              print("\nThe column is not numeric.")
  if display_print == True:
      print(f"\nName: {column_name}, dtype: {data[column_name].dtype}")


def return_unique_values(df, column_name):
    return df[column_name].unique()

In [ ]:
stats_unique_values(df_sales, 'transaction_id')

In [ ]:
stats_unique_values(df_sales, 'timestamp')

In [ ]:
stats_unique_values(df_sales, 'product_id')

In [ ]:
stats_unique_values(df_sales, 'category')

In [ ]:
stats_unique_values(df_sales, 'customer_type')

In [ ]:
stats_unique_values(df_sales, 'unit_price')

In [ ]:
stats_unique_values(df_sales, 'quantity')

In [ ]:
stats_unique_values(df_sales, 'total')

In [ ]:
stats_unique_values(df_sales, 'payment_type')

## ANALYZING STOCK INVENTORY

In [ ]:
#Stock
df_stock.head()

,id,timestamp,product_id,estimated_stock_pct
0,4220e505-c247-478d-9831-6b9f87a4488a,2022-03-07 12:13:02,f658605e-75f3-4fed-a655-c0903f344427,0.75
1,f2612b26-fc82-49ea-8940-0751fdd4d9ef,2022-03-07 16:39:46,de06083a-f5c0-451d-b2f4-9ab88b52609d,0.48
2,989a287f-67e6-4478-aa49-c3a35dac0e2e,2022-03-01 18:17:43,ce8f3a04-d1a4-43b1-a7c2-fa1b8e7674c8,0.58
3,af8e5683-d247-46ac-9909-1a77bdebefb2,2022-03-02 14:29:09,c21e3ba9-92a3-4745-92c2-6faef73223f7,0.79
4,08a32247-3f44-4002-85fb-c198434dd4bb,2022-03-02 13:46:18,7f478817-aa5b-44e9-9059-8045228c9eb0,0.22


In [ ]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  object 
 1   timestamp            15000 non-null  object 
 2   product_id           15000 non-null  object 
 3   estimated_stock_pct  15000 non-null  float64
dtypes: float64(1), object(3)
memory usage: 468.9+ KB


In [ ]:
df_stock.describe()

,estimated_stock_pct
count,15000.000000
mean,0.502735
std,0.286842
min,0.010000
25%,0.260000
50%,0.500000
75%,0.750000
max,1.000000


In [ ]:
stats_unique_values(df_stock, 'id')

In [ ]:
stats_unique_values(df_stock, 'timestamp')

In [ ]:
stats_unique_values(df_stock, 'product_id')

## ANALYZING TEMPERATURE IN REFRIGERATORS

In [ ]:
# Temperature
df_temperature.head()

,id,timestamp,temperature
0,d1ca1ef8-0eac-42fc-af80-97106efc7b13,2022-03-07 15:55:20,2.96
1,4b8a66c4-0f3a-4f16-826f-8cf9397e9d18,2022-03-01 09:18:22,1.88
2,3d47a0c7-1e72-4512-812f-b6b5d8428cf3,2022-03-04 15:12:26,1.78
3,9500357b-ce15-424a-837a-7677b386f471,2022-03-02 12:30:42,2.18
4,c4b61fec-99c2-4c6d-8e5d-4edd8c9632fa,2022-03-05 09:09:33,1.38


In [ ]:
df_temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23890 entries, 0 to 23889
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           23890 non-null  object 
 1   timestamp    23890 non-null  object 
 2   temperature  23890 non-null  float64
dtypes: float64(1), object(2)
memory usage: 560.0+ KB


In [ ]:
df_temperature.describe()

,temperature
count,23890.000000
mean,-0.207075
std,11.217649
min,-30.990000
25%,-2.860000
50%,-1.000000
75%,1.840000
max,34.990000


In [ ]:
stats_unique_values(df_temperature, 'id')

In [ ]:
stats_unique_values(df_temperature, 'timestamp')

In [ ]:
stats_unique_values(df_temperature, 'temperature')

## Convesion: Obj > Timestamp > Hourly timestamp

In [ ]:
from datetime import datetime

In [ ]:
def convert_to_datetime(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  dummy[column] = pd.to_datetime(dummy[column], format='%Y-%m-%d %H:%M:%S')
  return dummy

def convert_timestamp_to_hourly(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  new_ts = dummy[column].tolist()
  new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
  new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
  dummy[column] = new_ts
  return dummy

In [ ]:
#Sales

# convert to Timestamp
df_sales = convert_to_datetime(df_sales, 'timestamp')
# convert Timestamp to Hourly
df_sales = convert_timestamp_to_hourly(df_sales, 'timestamp')
# display
df_sales.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:00:00,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:00:00,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:00:00,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:00:00,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:00:00,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [ ]:
#Stock

# convert to Timestamp
df_stock = convert_to_datetime(df_stock, 'timestamp')
# convert Timestamp to Hourly
df_stock = convert_timestamp_to_hourly(df_stock, 'timestamp')
#display
df_stock.head()

,id,timestamp,product_id,estimated_stock_pct
0,4220e505-c247-478d-9831-6b9f87a4488a,2022-03-07 12:00:00,f658605e-75f3-4fed-a655-c0903f344427,0.75
1,f2612b26-fc82-49ea-8940-0751fdd4d9ef,2022-03-07 16:00:00,de06083a-f5c0-451d-b2f4-9ab88b52609d,0.48
2,989a287f-67e6-4478-aa49-c3a35dac0e2e,2022-03-01 18:00:00,ce8f3a04-d1a4-43b1-a7c2-fa1b8e7674c8,0.58
3,af8e5683-d247-46ac-9909-1a77bdebefb2,2022-03-02 14:00:00,c21e3ba9-92a3-4745-92c2-6faef73223f7,0.79
4,08a32247-3f44-4002-85fb-c198434dd4bb,2022-03-02 13:00:00,7f478817-aa5b-44e9-9059-8045228c9eb0,0.22


In [ ]:
# Temperature

# convert to Timestamp
df_temperature = convert_to_datetime(df_temperature, 'timestamp')
# convert Timestamp to Hourly
df_temperature = convert_timestamp_to_hourly(df_temperature, 'timestamp')
# display
df_temperature.head()

,id,timestamp,temperature
0,d1ca1ef8-0eac-42fc-af80-97106efc7b13,2022-03-07 15:00:00,2.96
1,4b8a66c4-0f3a-4f16-826f-8cf9397e9d18,2022-03-01 09:00:00,1.88
2,3d47a0c7-1e72-4512-812f-b6b5d8428cf3,2022-03-04 15:00:00,1.78
3,9500357b-ce15-424a-837a-7677b386f471,2022-03-02 12:00:00,2.18
4,c4b61fec-99c2-4c6d-8e5d-4edd8c9632fa,2022-03-05 09:00:00,1.38


Now all of the `timestamp` columns have had the minutes and seconds reduced to `00`. The next thing to do, is to aggregate the datasets in order to combine rows which have the same value for `timestamp`.


## Data Aggregation

1. For the `sales` data, I want to group the data by `timestamp` but also by `product_id`. During aggregation I must choose which columns to aggregate by the grouping.

In [ ]:
sales_agg = df_sales.groupby(['timestamp', 'product_id']).agg({'quantity': 'sum'}).reset_index()
sales_agg.head()

,timestamp,product_id,quantity
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,3
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,3
2,2022-03-01 09:00:00,03a2557a-aa12-4add-a6d4-77dc36342067,3
3,2022-03-01 09:00:00,049b2171-0eeb-4a3e-bf98-0c290c7821da,7
4,2022-03-01 09:00:00,04da844d-8dba-4470-9119-e534d52a03a0,11


I have now an aggregated sales data where each row represents a unique combination of hour during which the sales took place from that weeks worth of data and the product_id. I summed the quantity and I took the mean average of the unit_price.

2. For the stock data, we want to group it in the same way and aggregate the estimated_stock_pct.

In [ ]:
stock_agg = df_stock.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
stock_agg.head()

,timestamp,product_id,estimated_stock_pct
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05


This shows the average stock percentage of each product at unique hours within the week of sample data.

3. For the temperature data, product_id does not exist in this table, so we simply need to group by timestamp and aggregate the temperature.

In [ ]:
temp_agg = df_temperature.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()
temp_agg.head()

,timestamp,temperature
0,2022-03-01 09:00:00,-0.028850
1,2022-03-01 10:00:00,1.284314
2,2022-03-01 11:00:00,-0.560000
3,2022-03-01 12:00:00,-0.537721
4,2022-03-01 13:00:00,-0.188734


This gives me the average temperature of the storage facility where the produce is stored in the warehouse by unique hours during the week.


## Merge Data
Now, I am ready to merge our data. I will use the `stock_agg` table as our base table, and I will merge our other 2 tables onto this.

In [ ]:
# Merge Sales w/ Stock
merged_df = stock_agg.merge(sales_agg, on=['timestamp', 'product_id'], how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN


In [ ]:
# Merge Sales+Stock w/ temp_agg
merged_df = merged_df.merge(temp_agg, on='timestamp', how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN,-0.02885
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN,-0.02885
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN,-0.02885


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             3067 non-null   float64       
 4   temperature          10845 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 508.4+ KB


## Improve data quality

I can see from the .info() method that I have some null values.

 These need to be treated before I can build a predictive model. The column that features some null values is quantity.

 I can assume that if there is a null value for this column, it represents that there were 0 sales of this product within this hour.

 So, lets fill this columns null values with 0, however, I should verify this with the client, in order to make sure I'm not making any assumptions by filling these null values with 0.

In [ ]:
merged_df['quantity'] = merged_df['quantity'].fillna(0)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             10845 non-null  float64       
 4   temperature          10845 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 508.4+ KB


We can combine some more features onto this table too, including
`category` and `unit_price`.

In [ ]:
product_categories = df_sales[['product_id', 'category']]
product_categories = product_categories.drop_duplicates()

product_price = df_sales[['product_id', 'unit_price']]
product_price = product_price.drop_duplicates()

In [ ]:
# Add gategory
merged_df = merged_df.merge(product_categories, on="product_id", how="left")
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature,category
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,kitchen
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,vegetables
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,baby products
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,beverages
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,pets


In [ ]:
# Add product price
merged_df = merged_df.merge(product_price, on="product_id", how="left")
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature,category,unit_price
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,kitchen,11.19
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,vegetables,1.49
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,baby products,14.19
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,beverages,20.19
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,pets,8.19


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             10845 non-null  float64       
 4   temperature          10845 non-null  float64       
 5   category             10845 non-null  object        
 6   unit_price           10845 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 677.8+ KB


## Feature Enginering

I have cleaned and merged data. Now I transform this data so that the columns are in a suitable format for a ML model.. On other terms, every column must be numeric.

There are some models that will accept categorical features, but in this case we will use a model that requires numeric features

### > Date Enginering

Since it's a datetime datatype, I can explode this code into day of week, day of month and hour.

Looking at the latest table, I only have 1 remaining column which is not numeric. This is the `product_id`.

Since each row represents a unique combination of product_id and timestamp by hour, and the product_id is simply an ID column, it will add no value by including it in the predictive model. Hence, we shall remove it from the modeling process.

This feature engineering was by no means exhaustive, but was enough to give an example of the process followed when engineering the features of a dataset.

In reality, this is an iterative task. Once you've built a model, you may have to revist feature engineering in order to create new features to boost the predictive power of a machine learning model.

In [ ]:
column_name = 'timestamp'
if column_name in merged_df.columns:
    print(f'{column_name} exists in the DataFrame.')
    # timestamp > day of month
    merged_df['day'] = merged_df['timestamp'].dt.day
    # timestamp > day of week
    merged_df['weekday'] = merged_df['timestamp'].dt.dayofweek
    # timestamp > hour
    merged_df['hour'] = merged_df['timestamp'].dt.hour
    # delete timestamp
    merged_df.drop(columns=['timestamp'], inplace=True)
    # show merged_df

column_name = 'product_id'
if column_name in merged_df.columns:
    merged_df.drop(columns=[column_name], inplace=True)

merged_df

timestamp exists in the DataFrame.


,estimated_stock_pct,quantity,temperature,category,unit_price,day,weekday,hour
0,0.89,3.0,-0.028850,kitchen,11.19,1,1,9
1,0.14,3.0,-0.028850,vegetables,1.49,1,1,9
2,0.67,0.0,-0.028850,baby products,14.19,1,1,9
3,0.82,0.0,-0.028850,beverages,20.19,1,1,9
4,0.05,0.0,-0.028850,pets,8.19,1,1,9
...,...,...,...,...,...,...,...,...
10840,0.50,4.0,-0.165077,fruit,4.99,7,0,19
10841,0.26,0.0,-0.165077,meat,19.99,7,0,19
10842,0.78,3.0,-0.165077,packaged foods,6.99,7,0,19
10843,0.92,3.0,-0.165077,meat,14.99,7,0,19


In [ ]:
# Define the file path where you want to save the CSV file
file_path = '/content/my_dataframe.csv'

# Save the DataFrame to the CSV file
merged_df.to_csv(file_path, index=False)

# Verify that the file has been saved
print(f'DataFrame saved to {file_path}')

DataFrame saved to /content/my_dataframe.csv


The next column that I can engineer is the `category` column. In its current form it is categorical. I can convert it into numeric by creating dummy variables from this categorical column.

A dummy variable is a binary flag column (1's and 0's) that indicates whether a row fits a particular value of that column. For example, I can create a dummy column called category_pets, which will contain a 1 if that row indicates a product which was included within this category and a 0 if not.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor

from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.metrics import make_scorer, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, r2_score
from sklearn.metrics import roc_auc_score, log_loss, precision_score, recall_score, f1_score

from sklearn.compose import ColumnTransformer

import joblib

import time


# Model Building

Now it is time to train a ML model.



Now it is time to train a machine learning model. I will use a supervised machine learning model, and I will use estimated_stock_pct as the target variable, since the problem statement was focused on being able to predict the stock levels of products on an hourly basis.
Whilst training the machine learning model, I will use cross-validation, which is a technique where I hold back a portion of the dataset for testing in order to compute how well the trained machine learning model is able to predict the target variable.
Finally, to ensure that the trained machine learning model is able to perform robustly, I will want to test it several times on random samples of data, not just once. Hence, we will use a K-fold strategy to train the machine learning model on K (K is an integer to be decided) random samples of the data.
First, let's create our target variable y and independent variables X

In [ ]:
# Split data
X = merged_df.drop('estimated_stock_pct', axis=1)
y = merged_df['estimated_stock_pct']
print(X.shape)
print(y.shape)

(10845, 7)
(10845,)


This shows that I have 29 predictor variables that I will train our machine learning model on and 10845 rows of data.

Now let's define how many folds I want to complete during training, and how much of the dataset to assign to training, leaving the rest for test.

Typically, I should leave at least 20-30% of the data for testing.

In [ ]:
# Splitting for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Encode categorical features
to_normalize = ['quantity', 'temperature', 'unit_price', 'day', 'weekday', 'hour']
to_one_hot = ['category']
# Preprocessing
ct = make_column_transformer(
    (OneHotEncoder(), to_one_hot),
    (Normalizer(), to_normalize)
)

In [ ]:
# ML pipeline function
def get_model_info(model):
    info = {}
    # Check if the model is a classifier or regressor
    if hasattr(model, 'predict_proba') or hasattr(model, 'predict_log_proba'):
        info['ModelType'] = 'Classifier'
    else:
        info['ModelType'] = 'Regressor'

    # Get model parameters or hyperparameters
    info['Model Parameters'] = model.get_params()

    # Get model coefficients or feature importances if applicable
    if hasattr(model, 'coef_'):
        info['ModelCoefficients'] = model.coef_
    elif hasattr(model, 'feature_importances_'):
        info['ModelFeatureImportances'] = model.feature_importances_

    # Calculate the number of model features
    if hasattr(model, 'n_features_in_'):
        info['ModelNumberOfFeatures'] = model.n_features_in_

    return info
SCORER = 'neg_mean_absolute_error'
def ml_pipeline_cv(model, ct, X_train: pd.DataFrame, y_train: pd.Series, cv: int = 4, iterate: bool = False):
    """Creates a pipeline using a column transformer and model. K-fold cross validation
    on the pipeline.

    Args:
        model: Model compatible with sklearn Pipeline.
        ct: Sklearn column transformer.
        X_train (pd.DataFrame): Features dataframe.
        y_train (pd.Series): Target series.
        cv (int): K-fold cross validation.

    Returns:
        float: The average error selected.
    """
    # GET SCORERS
    scorers = {
        'regression': {
            'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
            'MAPE': make_scorer(mean_absolute_percentage_error, greater_is_better=False),
            'RMSLE': make_scorer(mean_squared_log_error, greater_is_better=False),
            'R^2': make_scorer(r2_score)
        },
        'classification': {
            'AUC': make_scorer(roc_auc_score),
            'LogLoss': make_scorer(log_loss, greater_is_better=False),
            'Precision': make_scorer(precision_score),
            'Recall': make_scorer(recall_score),
            'F1Score': make_scorer(f1_score)
        }
    }
    # PIPELINE SETTINGS
    pipe = Pipeline([
        ('ct', ct),
        ('model', model)
    ])
    # GENERATE RESULTS STORE
    results = {}
    results['ModelName'] = model
    results['TestedCrossValidation'] = []

    if(cv<=3):
        cv = 4

    if iterate == True:
      cv_init = 4
    else:
      cv_init = cv



    for K in range(cv_init, cv+1):
        iteration = {}
        start_time = time.time()
        iteration['CrossValidation'] = K
        ##
        # CHECK if MODEL is a Classifier or Regressor
        if hasattr(model, 'predict_proba') or hasattr(model, 'predict_log_proba'):
              # The model has predict_proba or predict_log_proba method, indicating it's a classifier
              for scorer_name, scorer in scorers['classification'].items():
                  cv_results = cross_validate(pipe, X_train, y_train, cv=K, scoring=scorer, error_score='raise')
                  results[scorer_name] = cv_results['test_score'].mean()
        else:
              # Assume it's a regressor if it doesn't have predict_proba or predict_log_proba
              for scorer_name, scorer in scorers['regression'].items():
                  cv_results = cross_validate(pipe, X_train, y_train, cv=K, scoring=scorer, error_score='raise')
                  results[scorer_name] = cv_results['test_score'].mean()
        ##
        iteration['TimeElapsed'] = (f"{(time.time() - start_time):.2f}")
        print(f"cv-{K} | Iteration Ended in {iteration['TimeElapsed']} s")
        results['TestedCrossValidation'].append(iteration)


    return results

In [ ]:
set_CrossValidation = 5

In [ ]:
# Linear Regression using Stochastic Gradient Descent
sgd = SGDRegressor()
results_1 = ml_pipeline_cv(sgd, ct, X_train, y_train, cv=8, iterate = True)
print(results_1)

cv-4 | Iteration Ended in 0.27 s
cv-5 | Iteration Ended in 0.33 s
cv-6 | Iteration Ended in 0.39 s
cv-7 | Iteration Ended in 0.44 s
cv-8 | Iteration Ended in 0.54 s
{'ModelName': SGDRegressor(), 'TestedCrossValidation': [{'CrossValidation': 4, 'TimeElapsed': '0.27'}, {'CrossValidation': 5, 'TimeElapsed': '0.33'}, {'CrossValidation': 6, 'TimeElapsed': '0.39'}, {'CrossValidation': 7, 'TimeElapsed': '0.44'}, {'CrossValidation': 8, 'TimeElapsed': '0.54'}], 'MAE': -0.2219222721501147, 'MAPE': -1.505967700502132, 'RMSLE': -0.032576434943018207, 'R^2': -0.01152063616656851}


In [ ]:
# Support Vector Machine
svr = SVR()
results_2 = ml_pipeline_cv(svr, ct, X_train, y_train, cv=set_CrossValidation)

cv-5 | Iteration Ended in 32.03 s


In [ ]:
# Random Forest
rf = RandomForestRegressor()
results_3 = ml_pipeline_cv(rf, ct, X_train, y_train, cv=set_CrossValidation)

cv-5 | Iteration Ended in 300.70 s


In [ ]:
# Soft Voting Regressor
estimators = [('sgd', sgd), ('svr', svr), ('rf', rf)]
vr = VotingRegressor(estimators=estimators)
results_4 = ml_pipeline_cv(vr, ct, X_train, y_train, cv=set_CrossValidation)

cv-5 | Iteration Ended in 325.49 s


In [ ]:
# Soft Voting Regressor
estimators = [('sgd', sgd), ('svr', svr)]
vr = VotingRegressor(estimators=estimators)
results_5 = ml_pipeline_cv(vr, ct, X_train, y_train, cv=set_CrossValidation)

cv-5 | Iteration Ended in 32.13 s


In [ ]:
print(results_5)

{'ModelName': VotingRegressor(estimators=[('sgd', SGDRegressor()), ('svr', SVR())]), 'TestedCrossValidation': [{'CrossValidation': 5, 'TimeElapsed': '32.13'}], 'MAE': -0.22180993627861648, 'MAPE': -1.498750277960616, 'RMSLE': -0.03257338986372223, 'R^2': -0.010944805164326077}


# Model Tuning

In [ ]:
# Tuning grid search function
def model_tuning_cv(model, ct, param_grid, X_train: pd.DataFrame, y_train: pd.Series):
    """Creates a pipeline using a column transformer and model. Searched through all
    specified parameters with K-fold cross validation on the pipeline.

    Args:
        model: Model compatible with sklearn Pipeline.
        ct: Sklearn column transformer.
        param_grid: Dict or list of dicts containing the parameters to search through.
        X_train (pd.DataFrame): Features dataframe.
        y_train (pd.Series): Target series.
    """
    # PIPELINE SETTINGS
    pipe = Pipeline([
        ('ct', ct),
        ('model', model)
    ])
    gs = GridSearchCV(pipe, param_grid, scoring=SCORER, n_jobs=5)
    gs.fit(X_train, y_train)
    print(gs.best_params_)

In [ ]:
!pip install matplotlib
!pip install numpy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### SGD Linear Regression

In [ ]:
# Grid search
params_sgd = [
    {
        'model__alpha': [0.1, 0.01, 0.001, 0.0001],
        'model__loss': ['squared_error', 'huber'],
        'model__penalty': ['l2', 'l1'],
        'model__max_iter': [500, 1000, 1500, 2000],
        'model__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    },
    {
        'model__alpha': [0.1, 0.01, 0.001, 0.0001],
        'model__loss': ['squared_error', 'huber'],
        'model__penalty': ['elasticnet'],
        'model__l1_ratio': [0.5, 0.15, 0.25],
        'model__max_iter': [500, 1000, 1500, 2000],
        'model__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
    }
]
model_tuning_cv(sgd, ct, params_sgd, X_train, y_train)

{'model__alpha': 0.1, 'model__learning_rate': 'constant', 'model__loss': 'squared_error', 'model__max_iter': 2000, 'model__penalty': 'l1'}


is performing a grid search for hyperparameter tuning on a Stochastic Gradient Descent (SGD) model using scikit-learn. It explores various combinations of hyperparameters to find the best set of values for the model. The hyperparameters being tuned include the alpha (regularization strength), loss function, penalty type, learning rate, and maximum iterations. The code evaluates different combinations of these hyperparameters to optimize the model's performance.

In [ ]:
# Finer tune
params_sgd_finer = {
    'model__alpha': [0.1],
    'model__l1_ratio': [0.6],
    'model__learning_rate': ['constant'],
    'model__loss': ['squared_error'],
    'model__max_iter': np.arange(350, 400),
    'model__penalty': ['elasticnet']
}
model_tuning_cv(sgd, ct, params_sgd_finer, X_train, y_train)

{'model__alpha': 0.1, 'model__l1_ratio': 0.6, 'model__learning_rate': 'constant', 'model__loss': 'squared_error', 'model__max_iter': 376, 'model__penalty': 'elasticnet'}


 a finer-tuned grid search for hyperparameter tuning on the same Stochastic Gradient Descent (SGD) model. However, this time, it focuses on a more limited set of hyperparameters with specific values. The hyperparameters being tuned in this step include a fixed alpha value, a fixed l1_ratio value, a constant learning rate, squared error loss function, and a range of values for the maximum number of iterations. This finer-tuning narrows down the search space to further optimize the model's performance with specific hyperparameter choices.

In [ ]:
# Tuned model
sgd_best = SGDRegressor(alpha=0.1, learning_rate='constant', max_iter=376, penalty='elasticnet', l1_ratio=0.6)
# perform
sdg_res = ml_pipeline_cv(sgd_best, ct, X_train, y_train, cv=5)
print(sdg_res)

cv-5 | Iteration Ended in 0.58 s
{'ModelName': SGDRegressor(alpha=0.1, l1_ratio=0.6, learning_rate='constant', max_iter=376,
             penalty='elasticnet'), 'TestedCrossValidation': [{'CrossValidation': 5, 'TimeElapsed': '0.58'}], 'MAE': -0.22302568193756983, 'MAPE': -1.4650275825922698, 'RMSLE': -0.03243082970087452, 'R^2': -0.029638484790644837}


In [ ]:
import joblib

# Specify the file path for saving the .pkl file
model_pkl_path = '/content/sgd_best_model.pkl'  # Replace with your desired file path

# Save the trained model to the .pkl file
joblib.dump(sdg_res, model_pkl_path)

print(f"Trained model saved as {model_pkl_path}")

Trained model saved as /content/sgd_best_model.pkl


# SVR

In [ ]:
# Parameters
params_svr = {
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'model__degree': np.arange(1, 12, 2),
    'model__gamma': ['scale', 'auto'],
    'model__coef0': [0, 0.5, 0.9],
    'model__C': [1.0, 3.0, 5.0]
}
model_tuning_cv(svr, ct, params_svr, X_train, y_train)

In [ ]:
# Finer tune
params_svr_finer = {
    'model__kernel': ['poly'],
    'model__degree': np.arange(1, 6),
    'model__gamma': ['auto'],
    'model__C': [1, 2]
}
model_tuning_cv(svr, ct, params_svr_finer, X_train, y_train)

In [ ]:
# Tuned model
svr_best = SVR(kernel='poly', C=1, degree=3, gamma='auto', )
# perform
svr_res = ml_pipeline_cv(svr_best, ct, X_train, y_train, cv=5)
print(svr_res)

### Linear SGD & SVR Voting Regressor

In [ ]:
# Random tuning grid search function
def random_model_tuning_cv(model, ct, params, X_train: pd.DataFrame, y_train: pd.Series, n_iter: int=10, random_state: int=123):
    """Creates a pipeline using a column transformer and model. Searches through a
    random sample of specified parameters with K-fold cross validation on the pipeline.

    Args:
        model: Model compatible with sklearn Pipeline.
        ct: Sklearn column transformer.
        params: Dict or list of dicts containing the parameters to search through.
        X_train (pd.DataFrame): Features dataframe.
        y_train (pd.Series): Target series.
        n_iter (int, optional): Number of search iterations. Defaults to 10.
        random_state (int, optional): Random state instance. Defaults to 123.
    """
    pipe = Pipeline([
        ('ct', ct),
        ('model', model)
    ])
    random_gs = RandomizedSearchCV(
        pipe, params, n_iter=n_iter, cv=4, n_jobs=5, scoring=SCORER, random_state=random_state)
    random_gs.fit(X_train, y_train)
    print(random_gs.best_params_)

In [ ]:
params_sgd_svr = [
    {
        'model__sgd__alpha': [0.1, 0.01, 0.001, 0.0001],
        'model__sgd__loss': ['squared_error', 'huber'],
        'model__sgd__penalty': ['l2', 'l1'],
        'model__sgd__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'model__svr__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__svr__degree': [1, 3, 5, 7, 9, 11, 13, 15],
        'model__svr__gamma': ['scale', 'auto'],
        'model__svr__C': np.arange(1, 12, 2)
    },
    {
        'model__sgd__alpha': [ 0.01, 0.001],
        'model__sgd__loss': ['squared_error', 'huber'],
        'model__sgd__penalty': ['elasticnet'],
        'model__sgd__l1_ratio': [0.2, 0.5, 0.7, 0.9],
        'model__sgd__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'model__svr__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__svr__degree': [1, 3, 5, 7, 9, 11, 13, 15],
        'model__svr__gamma': ['scale', 'auto'],
        'model__svr__C': np.arange(1, 12, 2)
    }
]
estimators = [
    ('sgd', SGDRegressor()),
    ('svr', SVR())
]
voting_reg = VotingRegressor(estimators, n_jobs=5)
random_model_tuning_cv(
    voting_reg, ct, params_sgd_svr, X_train, y_train, n_iter=500)

In [ ]:
# Finer tune
params_sgd_svr = {
    'model__sgd__alpha': [0.1, 0.3],
    'model__sgd__loss': ['squared_error'],
    'model__sgd__penalty': ['l1'],
    'model__sgd__learning_rate': ['constant'],
    'model__svr__kernel': ['poly'],
    'model__svr__degree': np.arange(10, 21),
    'model__svr__gamma': ['auto'],
    'model__svr__C': np.arange(1, 7)
}
voting_reg = VotingRegressor(estimators, n_jobs=5)
model_tuning_cv(voting_reg, ct, params_sgd_svr, X_train, y_train)

In [ ]:
# Calculate Results for Voting Regressor
voting_sgd_best = SGDRegressor(
    alpha=0.3, penalty='l1', learning_rate='constant')
voting_svr_best = SVR(
    kernel='poly', degree=19, gamma='auto', C=5)

estimators_best = [
    ('sgd', voting_sgd_best),
    ('svr', voting_svr_best)
]
voting_reg_best = VotingRegressor(estimators_best)

voting_res = ml_pipeline_cv(voting_reg_best, ct, X_train, y_train, cv=5)
print(voting_res)